<a href="https://colab.research.google.com/github/webgioant/Flask-Tree-Analysis/blob/main/SAGARA_SEKTOR_ARAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
SAGARA_ERA5_MULTIRIVA_NEXT.ipynb
(Versi yang dimodifikasi untuk integrasi GUI)
"""

# 1. IMPORTS
!pip install windrose openpyxl scipy netCDF4 h5netcdf

import os
import re
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from windrose import WindroseAxes
import openpyxl as ox
from openpyxl.styles import Alignment, Font
from scipy.stats import gumbel_r
from math import sqrt
import datetime # Import datetime for date parsing
import ipywidgets as widgets # <-- IMPORT BARU UNTUK GUI
from IPython.display import display, clear_output # <-- IMPORT BARU UNTUK GUI

#----------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------
# 2. DEFINISI KELAS (Seluruh kelas ERA5WindRose Anda disalin di sini)
class ERA5WindRose:
    def __init__(self, file_paths, lat, lon, start_date, end_date, convert_to_cms=False):
        # Modified to accept a list of file paths
        self.file_paths = file_paths
        self.lat = lat
        self.lon = lon
        self.start_date = start_date
        self.end_date = end_date
        self.convert_to_cms = convert_to_cms

        self.ds = None # Merged dataset
        self.subset = None # Subsetted dataset for the specific location and time range
        self.df = None
        self.table_counts = None
        self.table_percent = None
        self.wave_table_counts = None
        self.wave_table_percent = None
        self.speed_units = 'm/s' if not convert_to_cms else 'cm/s'
        self.dir_labels = ['N','NNE','NE','ENE','E','ESE','SE','SSE',
                      'S','SSW','SW','WSW','W','WNW','NW','NNW']
        self.dir_bins = np.linspace(-11.25, 360.0 + 11.25, len(self.dir_labels) + 1)
        self.speed_bins = [0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]
        self.speed_labels = ['0–2.5', '2.5–5.0', '5.0–7.5', '7.5–10.0',
                        '10.0–12.5', '12.5–15.0', '15.0–17.5', '17.5–20.0']
        self.wave_bins = None # Initialize as None
        self.wave_labels = None # Initialize as None


    def load_dataset(self, engine=None, fallback_search_radius=30):
        """Open datasets from the list, merge them, subset nearest grid and time.
           If swh empty at chosen grid, try to find nearest valid swh cell."""
        if not isinstance(self.file_paths, list) or not self.file_paths:
             raise ValueError("file_paths must be a non-empty list of file paths.")

        datasets = []
        for fpath in self.file_paths:
            if not os.path.exists(fpath):
                print(f"! File tidak ditemukan, dilewati: {fpath}")
                continue
            try:
                if engine:
                    ds = xr.open_dataset(fpath, engine=engine)
                else:
                    ds = xr.open_dataset(fpath)
                datasets.append(ds)
                print(f"Berhasil membuka dataset: {fpath}")
            except Exception as e:
                print(f"X Gagal membuka dataset, dilewati: {fpath} - {e}")

        if not datasets:
            raise RuntimeError("Tidak ada dataset yang berhasil dibuka dari daftar file.")

        # Merge datasets
        try:
            # Ensure datasets are sorted by time before concatenating
            datasets = sorted(datasets, key=lambda ds: ds['time'].values.min() if 'time' in ds.coords and ds['time'].size > 0 else pd.Timestamp.min)
            self.ds = xr.concat(datasets, dim='time', coords='minimal', compat='override')
            print("Berhasil menggabungkan dataset.")
        except Exception as e:
            raise RuntimeError(f"Gagal menggabungkan dataset: {e}")


        print("\n=== Merged Dataset variables (name : dims / shape) ===")
        for name, da in self.ds.data_vars.items():
            try:
                print(f"  - {name:20s} : {da.dims} / {tuple(da.shape)}")
            except Exception:
                print(f"  - {name:20s} : (tidak bisa tampilkan shape)")


        # find lat/lon coordinate names in the merged dataset
        lat_coord = None
        lon_coord = None
        for cand in ['latitude', 'lat', 'y']:
            if cand in self.ds.coords:
                lat_coord = cand
                break
        for cand in ['longitude', 'lon', 'x']:
            if cand in self.ds.coords:
                lon_coord = cand
                break
        if lat_coord is None or lon_coord is None:
            raise RuntimeError("Koordinat latitude/longitude tidak ditemukan di dataset gabungan (cari 'latitude'/'lat' dan 'longitude'/'lon').")

        lats = self.ds.coords[lat_coord].values
        lons = self.ds.coords[lon_coord].values
        lat_idx = np.abs(lats - self.lat).argmin()
        lon_idx = np.abs(lons - self.lon).argmin()

        # Subset the merged dataset based on the nearest grid cell and time range
        try:
            # Select nearest grid cell
            subset_location = self.ds.isel({lat_coord: lat_idx, lon_coord: lon_idx})
            # Select time slice
            self.subset = subset_location.sel(time=slice(self.start_date, self.end_date))

            # Update self.lat and self.lon to the exact coordinates of the selected grid cell
            self.lat = float(subset_location.coords[lat_coord].values)
            self.lon = float(subset_location.coords[lon_coord].values)
            print(f"Berhasil melakukan subset lokasi ke grid terdekat ({self.lat:.4f}, {self.lon:.4f}) dan waktu {self.start_date} s/d {self.end_date}.")

        except Exception as e:
            raise RuntimeError(f"Gagal melakukan subset lokasi/waktu dari dataset gabungan: {e}")


        # check u10/v10 existence in the subset
        if not all(v in self.subset.data_vars for v in ['u10', 'v10']):
            present = list(self.subset.data_vars.keys())
            raise ValueError(f"Dataset gabungan/subset tidak memiliki 'u10' dan 'v10'. Variables saat ini: {present}")

        # Check swh existence and validity in the subset
        if 'swh' in self.subset.data_vars:
            try:
                vals = np.squeeze(self.subset['swh'].values)
                finite_count = np.isfinite(vals).sum()
            except Exception:
                finite_count = 0

            if finite_count == 0:
                print("! swh di titik subset kosong/invalid. Mencari sel terdekat di dataset gabungan yang valid...")
                # Use the merged dataset for the fallback search
                res = find_nearest_valid_swh(self.ds, self.lat, self.lon, lat_coord, lon_coord, max_search=fallback_search_radius)
                if res is None:
                    print("X Tidak ditemukan sel terdekat dengan swh valid dalam radius pencarian di dataset gabungan.")
                    # Keep the subset even if swh is invalid, other variables might be ok
                    pass
                else:
                    i, j, dist = res
                    # update subset to that valid swh grid cell from the merged dataset
                    subset_location_valid = self.ds.isel({lat_coord: i, lon_coord: j})
                    self.subset = subset_location_valid.sel(time=slice(self.start_date, self.end_date))
                    # update lat/lon to the coordinate values of the valid swh cell
                    self.lat = float(self.ds.coords[lat_coord].values[i])
                    self.lon = float(self.ds.coords[lon_coord].values[j])
                    print(f"--> Berpindah ke grid terdekat dengan swh valid (idx {i},{j}, jarak ~{dist:.4f} deg) di dataset gabungan.")
            else:
                print("Variabel 'swh' ditemukan di subset dan memiliki nilai valid.")
        else:
            print("! Variabel 'swh' tidak ditemukan langsung di dataset gabungan/subset. Akan dicari nama alternatif saat extract wave height.")


    def compute_wind(self):
        if self.subset is None:
            raise RuntimeError("Panggil load_dataset() terlebih dahulu.")

        u10_da = self.subset['u10']
        v10_da = self.subset['v10']
        if 'time' not in u10_da.dims:
            raise RuntimeError("Variabel u10/v10 tidak memiliki dimensi 'time'.")

        times = pd.to_datetime(u10_da['time'].values)
        u10 = np.squeeze(u10_da.values)
        v10 = np.squeeze(v10_da.values)

        speed_mps = np.sqrt(u10**2 + v10**2)
        direction_deg = (180.0 / np.pi) * np.arctan2(-u10, -v10)
        direction_deg = (direction_deg + 360.0) % 360.0

        df = pd.DataFrame({'time': times, 'speed_mps': speed_mps, 'direction_deg': direction_deg})

        if len(df) >= 2:
            diffs = df['time'].diff().dropna().dt.total_seconds()
            median_sec = float(diffs.median()) if len(diffs) > 0 else 3600.0
            hours_per_sample = 1.0 if median_sec <= 0 or not np.isfinite(median_sec) else median_sec / 3600.0
        else:
            hours_per_sample = 1.0
        df['hours'] = hours_per_sample

        mask = np.isfinite(df['speed_mps']) & np.isfinite(df['direction_deg'])
        df = df.loc[mask].reset_index(drop=True)
        if df.shape[0] == 0:
            raise ValueError("Tidak ada data kecepatan/arah angin yang valid pada rentang waktu/lokasi ini.")

        if self.convert_to_cms:
            df['speed'] = df['speed_mps'] * 100.0
            self.speed_units = 'cm/s'
        else:
            df['speed'] = df['speed_mps']
            self.speed_units = 'm/s'
        df['direction'] = df['direction_deg']

        self.df = df

    def compute_waveheight(self):
        if self.subset is None:
            raise RuntimeError("Panggil load_dataset() terlebih dahulu.")
        if self.df is None:
            raise RuntimeError("Panggil compute_wind() terlebih dahulu.")

        candidates = list(self.subset.data_vars.keys())
        selected_var = None
        if 'swh' in candidates:
            selected_var = 'swh'
        else:
            patterns = ['swh', r'\bhs\b', 'hsig', 'wave', 'height']
            for pat in patterns:
                for cand in candidates:
                    if re.search(pat, cand, flags=re.IGNORECASE):
                        selected_var = cand
                        break
                if selected_var:
                    break

        if selected_var is None:
            print("! Tidak menemukan variable wave height (swh/hs/...) di subset. Variable tersedia:", candidates)
            self.df['swh'] = np.nan
            return

        da = self.subset[selected_var]
        print(f"Menemukan wave variable: '{selected_var}' dengan dims {da.dims} dan shape {tuple(da.shape)}")

        try:
            arr = np.squeeze(da.values)
        except Exception:
            arr = da.values
        arr_flat = np.array(arr).ravel()
        # treat fill/sentinel values as invalid: use attr or encoding if present
        fill = da.attrs.get('_FillValue', None) or da.encoding.get('_FillValue', None) or da.attrs.get('missing_value', None) or da.encoding.get('missing_value', None)
        if fill is not None:
            valid_mask = np.isfinite(arr_flat) & (np.abs(arr_flat - fill) > 0)
        else:
            valid_mask = np.isfinite(arr_flat) & (np.abs(arr_flat) < 1e30)
        valid_count = int(np.sum(valid_mask))
        print(f"Wave data size raw: {arr_flat.size}, valid (after masking): {valid_count}")

        if valid_count == 0:
            print("! Semua nilai wave height tidak valid pada titik ini.")
            self.df['swh'] = np.nan
            return

        # attempt align by time if lengths match or if da has time coord
        if arr_flat.size == len(self.df):
            self.df['swh'] = arr_flat
        else:
            if 'time' in da.coords:
                times_da = pd.to_datetime(da['time'].values)
                df_swh = pd.DataFrame({'time': times_da, 'swh': arr_flat})
                merged = pd.merge(self.df, df_swh, on='time', how='left')
                self.df = merged
            else:
                self.df['swh'] = np.nan
                print("! Wave variable tidak sejajar dengan time wind data; swh diset NaN.")
                return


        swh_valid = self.df['swh'].dropna().values
        if swh_valid.size > 0:
            print(f"Wave height dibaca: {selected_var} — jumlah sampel valid: {swh_valid.size}")
            print(f"   min={np.nanmin(swh_valid):.4f}, max={np.nanmax(swh_valid):.4f}, mean={np.nanmean(swh_valid):.4f}")
        else:
            print("! Setelah align, tidak ditemukan nilai swh yang valid pada rentang waktu/lokasi ini.")
            self.df['swh'] = np.nan


    def generate_statistics(self):
        if self.df is None:
            raise RuntimeError("Data belum dihitung. Panggil compute_wind() dulu.")
        if self.df.empty:
            print("! DataFrame kosong (mungkin karena filter sektoral). Statistik angin tidak dibuat.")
            self.table_counts = pd.DataFrame()
            self.table_percent = pd.DataFrame()
            return

        df = self.df.copy()
        df['dir_bin'] = pd.cut(df['direction'], bins=self.dir_bins, labels=self.dir_labels, right=False, include_lowest=True)
        df['speed_bin'] = pd.cut(df['speed'], bins=self.speed_bins, labels=self.speed_labels, right=False, include_lowest=True)

        table_counts = pd.crosstab(df['dir_bin'], df['speed_bin'], values=df['hours'], aggfunc='sum', dropna=False).fillna(0)
        table_counts['Total'] = table_counts.sum(axis=1)
        total_sum = table_counts.sum(axis=0)
        table_counts.loc['Total'] = total_sum

        table_percent = table_counts.copy()
        row_totals = table_counts['Total'].replace(0, np.nan)
        for col in table_percent.columns[:-1]:
            table_percent[col] = (table_percent[col] / row_totals * 100).round(2)

        grand_total = table_counts.loc['Total','Total']
        if grand_total > 0:
            table_percent['Total'] = (table_counts['Total'] / grand_total * 100).round(2)
        else:
            table_percent['Total'] = 0.0

        self.table_counts = table_counts
        self.table_percent = table_percent


    def generate_wave_statistics(self):
        if self.df is None:
            raise RuntimeError("Data belum dihitung. Panggil compute_wind() dulu.")
        if 'swh' not in self.df.columns or self.df['swh'].dropna().empty:
             print("! Tidak ada data 'swh' yang valid untuk membuat statistik wave height.")
             self.wave_table_counts = None
             self.wave_table_percent = None
             return

        # Define wave height bins (adjust as needed)
        self.wave_bins = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, np.inf] # Assign to self
        self.wave_labels = ['0-0.5', '0.5-1.0', '1.0-1.5', '1.5-2.0', '2.0-2.5', '2.5-3.0', '3.0-4.0', '4.0-5.0', '>5.0'] # Assign to self

        df = self.df.copy()
        df['wave_bin'] = pd.cut(df['swh'], bins=self.wave_bins, labels=self.wave_labels, right=False, include_lowest=True)

        # Use wind direction as a proxy for wave direction by default
        wave_direction = df['direction']

        # Check if mwd is available in the *subset* (only if subset exists)
        if self.subset is not None and 'mwd' in self.subset.data_vars:
             try:
                mwd_da = self.subset['mwd']
                mwd_arr = np.squeeze(mwd_da.values)
                mwd_flat = np.array(mwd_arr).ravel()
                # treat fill/sentinel values as invalid
                fill = mwd_da.attrs.get('_FillValue', None) or mwd_da.encoding.get('_FillValue', None) or mwd_da.attrs.get('missing_value', None) or mwd_da.encoding.get('missing_value', None)
                if fill is not None:
                    valid_mask_mwd = np.isfinite(mwd_flat) & (np.abs(mwd_flat - fill) > 0)
                else:
                    valid_mask_mwd = np.isfinite(mwd_flat) & (np.abs(mwd_flat) < 1e30)

                if np.sum(valid_mask_mwd) > 0:
                     # Align mwd with time if necessary
                    if mwd_flat.size == len(self.df):
                         df['mwd'] = mwd_flat
                    elif 'time' in mwd_da.coords:
                        times_mwd = pd.to_datetime(mwd_da['time'].values)
                        df_mwd = pd.DataFrame({'time': times_mwd, 'mwd': mwd_flat})
                        merged = pd.merge(df, df_mwd, on='time', how='left')
                        df = merged # Update df with merged data

                    # Use mwd if it was successfully merged and has valid data
                    if 'mwd' in df.columns and df['mwd'].dropna().size > 0:
                        wave_direction = df['mwd']
                        print("Menggunakan mean wave direction (mwd) untuk statistik gelombang.")
                    else:
                         print("! mwd variable tidak sejajar dengan time data; menggunakan arah angin untuk statistik gelombang.")
                else:
                    print("! mwd variable ada di subset tetapi semua nilai tidak valid; menggunakan arah angin untuk statistik gelombang.")

             except Exception as e:
                print(f"! Error saat memproses mwd untuk statistik gelombang: {e}. Menggunakan arah angin.")
        # Else: subset is None or mwd not in subset. Check if mwd is already in df (from combined data)
        elif 'mwd' in df.columns and df['mwd'].dropna().size > 0:
             wave_direction = df['mwd']
             print("Menggunakan mean wave direction (mwd) dari DataFrame gabungan untuk statistik gelombang.")
        else:
             print("! Variabel mwd tidak ditemukan atau valid. Menggunakan arah angin untuk statistik gelombang.")


        df['dir_bin'] = pd.cut(wave_direction, bins=self.dir_bins if hasattr(self, 'dir_bins') else np.linspace(-11.25, 360.0 + 11.25, 17), labels=self.dir_labels if hasattr(self, 'dir_labels') else ['N','NNE','NE','ENE','E','ESE','SE','SSE', 'S','SSW','SW','WSW','W','WNW','NW','NNW'], right=False, include_lowest=True)


        # Create a cross-tabulation of wave height bins and direction bins
        wave_table_counts = pd.crosstab(df['dir_bin'], df['wave_bin'], values=df['hours'], aggfunc='sum', dropna=False).fillna(0)

        # Ensure all direction labels are present, even if no data
        all_dir_labels = self.dir_labels if hasattr(self, 'dir_labels') else ['N','NNE','NE','ENE','E','ESE','SE','SSE', 'S','SSW','SW','WSW','W','WNW','NW','NNW']
        wave_table_counts = wave_table_counts.reindex(all_dir_labels, fill_value=0)

        # Calculate row totals
        wave_table_counts['Total'] = wave_table_counts.sum(axis=1)

        # Calculate column totals (including the new 'Total' row)
        total_sum = wave_table_counts.sum(axis=0)
        wave_table_counts.loc['Total'] = total_sum.values # Assign values directly

        wave_table_percent = wave_table_counts.copy()
        row_totals = wave_table_counts['Total'].replace(0, np.nan)
        # Calculate percentages based on row totals, exclude the final 'Total' row for this
        for col in wave_table_percent.columns[:-1]:
            wave_table_percent[col] = (wave_table_percent[col] / row_totals * 100).round(2)

        # Calculate percentage for the 'Total' column based on the overall total
        grand_total = wave_table_counts.loc['Total', 'Total']
        if grand_total > 0:
             wave_table_percent['Total'] = (wave_table_counts['Total'] / grand_total * 100).round(2)
        else:
             wave_table_percent['Total'] = 0.0

        # Calculate percentages for the 'Total' row based on the overall total
        if grand_total > 0:
            for col in wave_table_percent.columns[:-1]:
                 wave_table_percent.loc['Total', col] = (wave_table_counts.loc['Total', col] / grand_total * 100).round(2)
        else:
             wave_table_percent.loc['Total'] = 0.0


        self.wave_table_counts = wave_table_counts
        self.wave_table_percent = wave_table_percent
        print("Wave height statistics generated.")


    def save_to_excel(self, output_path):
        if self.table_counts is None or self.table_percent is None:
            raise RuntimeError("Statistik angin belum dibuat. Panggil generate_statistics() dulu.")
        if self.table_counts.empty:
            print(f"! Statistik angin kosong. Excel '{output_path}' tidak akan dibuat/diupdate.")
            return

        folder = os.path.dirname(output_path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)

        wb = ox.Workbook()
        sheet_wind = wb.active
        sheet_wind.title = "Wind Statistics"

        # Write Wind Statistics
        sheet_wind.merge_cells("A1:A3")
        sheet_wind["A1"] = "Direction"
        sheet_wind["A1"].alignment = Alignment(horizontal='center', vertical='center')
        sheet_wind["A1"].font = Font(bold=True)

        sheet_wind.merge_cells("B1:J1")
        sheet_wind["B1"] = "Number of Hours"
        sheet_wind["B1"].alignment = Alignment(horizontal='center')
        sheet_wind["B1"].font = Font(bold=True)

        sheet_wind.merge_cells("K1:S1")
        sheet_wind["K1"] = "Percentage"
        sheet_wind["K1"].alignment = Alignment(horizontal='center')
        sheet_wind["K1"].font = Font(bold=True)

        sheet_wind.merge_cells("B2:J2")
        sheet_wind["B2"] = f"Wind Speed ({self.speed_units})"
        sheet_wind["B2"].alignment =  Alignment(horizontal='center')
        sheet_wind["B2"].font = Font(bold=True)
        sheet_wind.merge_cells("K2:S2")
        sheet_wind["K2"] = f"Wind Speed ({self.speed_units})"
        sheet_wind["K2"].alignment =  Alignment(horizontal='center')
        sheet_wind["K2"].font = Font(bold=True)

        col_winds = self.speed_labels + ['Total']

        for i, c in enumerate(col_winds):
            sheet_wind.cell(row=3, column=i+2, value=c)
            sheet_wind.cell(row=3, column=i+11, value=c)

        for i, direction in enumerate(self.table_counts.index):
            sheet_wind.cell(row=4+i, column=1, value=str(direction))
            for j, val in enumerate(self.table_counts.loc[direction]):
                sheet_wind.cell(row=4+i, column=2+j, value=float(val))
            for j, val in enumerate(self.table_percent.loc[direction]):
                sheet_wind.cell(row=4+i, column=11+j, value=float(val))

        # Write Wave Statistics
        if self.wave_table_counts is not None and self.wave_table_percent is not None and not self.wave_table_counts.empty:
            sheet_wave = wb.create_sheet("Wave Statistics")

            sheet_wave.merge_cells("A1:A3")
            sheet_wave["A1"] = "Direction"
            sheet_wave["A1"].alignment = Alignment(horizontal='center', vertical='center')
            sheet_wave["A1"].font = Font(bold=True)

            # Adjust column merge if needed based on number of wave bins + total
            # Ensure wave_labels is defined
            if self.wave_labels is not None:
                num_wave_cols = len(self.wave_labels) + 1 # +1 for Total column
            else:
                 num_wave_cols = 1 # Just Total column if no wave labels
            start_col_counts = 2
            end_col_counts = start_col_counts + num_wave_cols - 1
            # Increase the gap between the two tables
            gap_cols = 2 # Number of empty columns between tables
            start_col_percent = end_col_counts + gap_cols + 1
            end_col_percent = start_col_percent + num_wave_cols - 1

            sheet_wave.merge_cells(start_row=1, start_column=start_col_counts, end_row=1, end_column=end_col_counts)
            sheet_wave.cell(row=1, column=start_col_counts, value="Number of Hours").alignment = Alignment(horizontal='center')
            sheet_wave.cell(row=1, column=start_col_counts).font = Font(bold=True)

            sheet_wave.merge_cells(start_row=1, start_column=start_col_percent, end_row=1, end_column=end_col_percent)
            sheet_wave.cell(row=1, column=start_col_percent, value="Percentage").alignment = Alignment(horizontal='center')
            sheet_wave.cell(row=1, column=start_col_percent).font = Font(bold=True)


            sheet_wave.merge_cells(start_row=2, start_column=start_col_counts, end_row=2, end_column=end_col_counts)
            sheet_wave.cell(row=2, column=start_col_counts, value="Wave Height (m)").alignment =  Alignment(horizontal='center')
            sheet_wave.cell(row=2, column=start_col_counts).font = Font(bold=True)

            sheet_wave.merge_cells(start_row=2, start_column=start_col_percent, end_row=2, end_column=end_col_percent)
            sheet_wave.cell(row=2, column=start_col_percent, value="Wave Height (m)").alignment =  Alignment(horizontal='center')
            sheet_wave.cell(row=2, column=start_col_percent).font = Font(bold=True)

            col_waves = (self.wave_labels if self.wave_labels is not None else []) + ['Total']

            for i, c in enumerate(col_waves):
                sheet_wave.cell(row=3, column=i+start_col_counts, value=c)
                sheet_wave.cell(row=3, column=i+start_col_percent, value=c)

            for i, direction in enumerate(self.wave_table_counts.index):
                sheet_wave.cell(row=4+i, column=1, value=str(direction))
                for j, val in enumerate(self.wave_table_counts.loc[direction]):
                    sheet_wave.cell(row=4+i, column=start_col_counts+j, value=float(val))
                for j, val in enumerate(self.wave_table_percent.loc[direction]):
                     sheet_wave.cell(row=4+i, column=start_col_percent+j, value=float(val))


        wb.save(output_path)
        print(f"Saved Excel: {output_path}")


    def plot_wind_rose(self, output_path):
        if self.df is None:
            raise RuntimeError("Data angin belum dihitung. Panggil compute_wind() dulu.")
        if self.df.empty:
            print(f"! DataFrame angin kosong. Windrose '{output_path}' tidak dibuat.")
            return

        bins = self.speed_bins
        fig = plt.figure(figsize=(8, 6))
        ax = WindroseAxes.from_ax(fig=fig)
        ax.bar(self.df['direction'], self.df['speed'], normed=True, opening=0.8,
               edgecolor='white', bins=bins, cmap=plt.cm.viridis)
        # Updated legend placement
        ax.set_legend(
            title=f"Wind Speed ({self.speed_units})",
            loc='upper right',
            bbox_to_anchor=(-0.25, 1),
            frameon=False
        )
        plt.title("Hourly Distribution of Wind Speed and Direction", pad=30, fontweight='bold')
        folder = os.path.dirname(output_path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
        plt.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved windrose: {output_path}")

    def plot_wave_rose(self, output_path):
        if self.df is None:
            raise RuntimeError("Data belum dihitung. Panggil compute_wind() dan compute_waveheight() dulu.")
        if 'swh' not in self.df.columns or self.df['swh'].dropna().empty:
             print("! Tidak ada data 'swh' yang valid untuk membuat waverose.")
             return
        if self.wave_bins is None or self.wave_labels is None:
             print("! Statistik gelombang belum dibuat. Panggil generate_wave_statistics() dulu.")
             return
        if self.df.empty:
            print(f"! DataFrame wave kosong. Waverose '{output_path}' tidak dibuat.")
            return


        # Use wind direction as a proxy for wave direction by default
        wave_direction = self.df['direction']

        # Check if mwd is available in the *subset* (only if subset exists)
        if self.subset is not None and 'mwd' in self.subset.data_vars:
            try:
                mwd_da = self.subset['mwd']
                mwd_arr = np.squeeze(mwd_da.values)
                mwd_flat = np.array(mwd_arr).ravel()
                # treat fill/sentinel values as invalid
                fill = mwd_da.attrs.get('_FillValue', None) or mwd_da.encoding.get('_FillValue', None) or mwd_da.attrs.get('missing_value', None) or mwd_da.encoding.get('missing_value', None)
                if fill is not None:
                    valid_mask_mwd = np.isfinite(mwd_flat) & (np.abs(mwd_flat - fill) > 0)
                else:
                    valid_mask_mwd = np.isfinite(mwd_flat) & (np.abs(mwd_flat) < 1e30)

                if np.sum(valid_mask_mwd) > 0:
                     # Align mwd with time if necessary
                    if mwd_flat.size == len(self.df):
                         self.df['mwd'] = mwd_flat
                    elif 'time' in mwd_da.coords:
                        times_mwd = pd.to_datetime(mwd_da['time'].values)
                        df_mwd = pd.DataFrame({'time': times_mwd, 'mwd': mwd_flat})
                        merged = pd.merge(self.df, df_mwd, on='time', how='left')
                        self.df = merged # Update self.df with merged data

                    # Use mwd if it was successfully merged and has valid data
                    if 'mwd' in self.df.columns and self.df['mwd'].dropna().size > 0:
                        wave_direction = self.df['mwd']
                        print("Menggunakan mean wave direction (mwd) untuk waverose.")
                    else:
                         print("! mwd variable tidak sejajar dengan time data; menggunakan arah angin untuk waverose.")
                else:
                    print("! mwd variable ada di subset tetapi semua nilai tidak valid; menggunakan arah angin untuk waverose.")

            except Exception as e:
                print(f"! Error saat memproses mwd: {e}. Menggunakan arah angin untuk waverose.")
        # Else: subset is None or mwd not in subset. Check if mwd is already in df (from combined data)
        elif 'mwd' in self.df.columns and self.df['mwd'].dropna().size > 0:
             wave_direction = self.df['mwd']
             print("Menggunakan mean wave direction (mwd) dari DataFrame gabungan untuk waverose.")
        else:
             print("! Variabel mwd tidak ditemukan atau valid. Menggunakan arah angin untuk waverose.")


        fig = plt.figure(figsize=(8, 6))
        ax = WindroseAxes.from_ax(fig=fig)
        # Use wave height as speed and wave direction
        ax.bar(wave_direction, self.df['swh'], normed=True, opening=0.8,
               edgecolor='white', bins=self.wave_bins, cmap=plt.cm.viridis) # Use self.wave_bins
        # --- Legend di luar tepi ---
        legend = ax.set_legend(
            title="Significant Wave Height (m)",
            loc='upper right',              # posisi relatif terhadap bbox anchor
            bbox_to_anchor=(-0.25, 1),      # (x, y): geser ke kiri di luar tepi
            frameon=False                   # tanpa border legend
        )
        plt.title("Hourly Distribution of Significant Wave Height and Direction", pad=30, fontweight='bold')
        folder = os.path.dirname(output_path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
        plt.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved waverose: {output_path}")


    def plot_gumbel_return_values(self, output_prefix):
        if self.df is None:
            raise RuntimeError("Data belum dihitung. Panggil compute_wind() dulu.")
        T = np.logspace(0, 2, 100)
        F = 1.0 - 1.0 / T

        folder = os.path.dirname(output_prefix)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)

        wind_data = self.df['speed'].dropna().values
        if wind_data.size >= 3:
            mu_ws, beta_ws = gumbel_r.fit(wind_data)
            x_ws = gumbel_r.ppf(F, loc=mu_ws, scale=beta_ws)
            plt.figure()
            plt.semilogx(T, x_ws, label='Gumbel Model')
            # Calculate plotting positions for empirical data (using Weibull or Gringorten)
            # Using Gringorten's method: (i - 0.44) / (n + 0.12)
            n_ws = wind_data.size
            ranks_ws = np.arange(1, n_ws + 1)
            plotting_positions_ws = (ranks_ws - 0.44) / (n_ws + 0.12)
            empirical_return_periods_ws = 1.0 / (1.0 - plotting_positions_ws)

            plt.scatter(empirical_return_periods_ws, np.sort(wind_data), facecolors='none', edgecolors='r', label='Data (sorted)')
            plt.xlabel('Return period (years)')
            plt.ylabel(f'Return value ({self.speed_units})')
            plt.title('Return values of Wind Speed in the Gumbel model', fontweight='bold')
            plt.legend()
            plt.grid(True, which="both", ls="--", lw=0.5)
            plt.savefig(f"{output_prefix}_wind_speed.png", dpi=300, bbox_inches="tight")
            plt.close()
            print(f"Saved Gumbel wind speed: {output_prefix}_wind_speed.png")
        else:
            print("! Tidak cukup data untuk fitting Gumbel wind speed (butuh >= 3 sampel).")

        wave_values = self.df.get('swh', pd.Series(dtype=float)).dropna().values
        if wave_values.size >= 3:
            mu_hs, beta_hs = gumbel_r.fit(wave_values)
            x_hs = gumbel_r.ppf(F, loc=mu_hs, scale=beta_hs)
            plt.figure()
            plt.semilogx(T, x_hs, label='Gumbel Model')
            # Calculate plotting positions for empirical data
            n_hs = wave_values.size
            ranks_hs = np.arange(1, n_hs + 1)
            plotting_positions_hs = (ranks_hs - 0.44) / (n_hs + 0.12)
            empirical_return_periods_hs = 1.0 / (1.0 - plotting_positions_hs)

            plt.scatter(empirical_return_periods_hs, np.sort(wave_values), facecolors='none', edgecolors='r', label='Data (sorted)')
            plt.xlabel('Return period (years)')
            plt.ylabel('Return value (m)')
            plt.title('Return values of Wave Height in the Gumbel model', fontweight='bold')
            plt.legend()
            plt.grid(True, which="both", ls="--", lw=0.5)
            plt.savefig(f"{output_prefix}_wave_height.png", dpi=300, bbox_inches="tight")
            plt.close()
            print(f"Saved Gumbel wave height: {output_prefix}_wave_height.png")
        else:
            print("! Tidak ada/kurang data 'swh' untuk analisis wave height (butuh >=3 sampel).")


    #-----Nilai Extreme Wind Wave----------
    def save_extreme_values_to_new_excel(self, output_path):
        """Simpan hasil Extreme Wind dan Wave ke file Excel baru."""
        Rp = np.array([1, 2, 5, 10, 25, 50, 100])
        F = 1 - 1 / Rp

        wind_data = self.df['speed'].dropna().values if self.df is not None and not self.df.empty else np.array([])
        wave_data = self.df['swh'].dropna().values if self.df is not None and 'swh' in self.df.columns and not self.df.empty else np.array([])

        # ===== Fit Gumbel untuk Wind =====
        if wind_data.size >= 3:
            mu_ws, beta_ws = gumbel_r.fit(wind_data)
            Ws = gumbel_r.ppf(F, loc=mu_ws, scale=beta_ws)
        else:
            if wind_data.size > 0:
                 print("! Tidak cukup data angin (butuh >= 3) untuk Gumbel di Excel.")
            Ws = np.full_like(Rp, np.nan, dtype=float)

        # ===== Fit Gumbel untuk Wave =====
        if wave_data.size >= 3:
            mu_hs, beta_hs = gumbel_r.fit(wave_data)
            Hs = gumbel_r.ppf(F, loc=mu_hs, scale=beta_hs)
            # Check for potential NaNs before calculating Tp
            Tp = 3.3 * np.sqrt(Hs) if np.isfinite(Hs).all() else np.full_like(Rp, np.nan, dtype=float)
        else:
            if wave_data.size > 0:
                 print("! Tidak cukup data wave (butuh >= 3) untuk Gumbel di Excel.")
            Hs = np.full_like(Rp, np.nan, dtype=float)
            Tp = np.full_like(Rp, np.nan, dtype=float)

        # ===== Tulis ke Excel baru =====
        wb = ox.Workbook()
        sheet = wb.active
        sheet.title = "Extreme Values"

        # Judul tabel
        sheet.merge_cells("A1:B1")
        sheet["A1"] = "Extreme Wind"
        sheet["A1"].alignment = Alignment(horizontal='center')
        sheet["A1"].font = Font(bold=True)

        sheet["A2"] = "RP [year]"
        sheet["B2"] = f"Ws [{self.speed_units}]"

        for i, (rp, ws) in enumerate(zip(Rp, Ws), start=3):
            sheet[f"A{i}"] = int(rp)
            sheet[f"B{i}"] = float(ws) if np.isfinite(ws) else None

        # Spasi antar tabel
        start_row = len(Rp) + 5

        sheet.merge_cells(f"A{start_row}:C{start_row}")
        sheet[f"A{start_row}"] = "Extreme Wave"
        sheet[f"A{start_row}"].alignment = Alignment(horizontal='center')
        sheet[f"A{start_row}"].font = Font(bold=True)

        sheet[f"A{start_row+1}"] = "RP [year]"
        sheet[f"B{start_row+1}"] = "Hs [m]"
        sheet[f"C{start_row+1}"] = "Tp [s]"

        for i, (rp, hs, tp) in enumerate(zip(Rp, Hs, Tp), start=start_row+2):
            sheet[f"A{i}"] = int(rp)
            sheet[f"B{i}"] = float(hs) if np.isfinite(hs) else None
            sheet[f"C{i}"] = float(tp) if np.isfinite(tp) else None

        wb.save(output_path)
        print(f"Extreme values saved to new Excel file: {output_path}")

#-------------------------------------------------------------------------------------------------------------------------
# 3. DEFINISI FUNGSI HELPER (find_nearest_valid_swh disalin di sini)
def find_nearest_valid_swh(ds, target_lat, target_lon, lat_coord, lon_coord, max_search=30):
    """Finds the nearest grid cell with valid swh data within a search radius."""
    lats = ds.coords[lat_coord].values
    lons = ds.coords[lon_coord].values

    # Find initial nearest index
    lat_idx_init = np.abs(lats - target_lat).argmin()
    lon_idx_init = np.abs(lons - target_lon).argmin()

    # Get swh data array
    swh_da = None
    # Check for 'swh' first
    if 'swh' in ds.data_vars:
        swh_da = ds['swh']
    else:
        # If 'swh' not found, search for other potential swh variables
        patterns = ['swh', r'\bhs\b', 'hsig', 'wave', 'height']
        for pat in patterns:
                for cand in ds.data_vars.keys():
                    if re.search(pat, cand, flags=re.IGNORECASE):
                        swh_da = ds[cand]
                        break
                if swh_da is not None:
                    break

    if swh_da is None:
        print("! Variabel swh (atau alternatif) tidak ditemukan di dataset.")
        return None

    # Handle potential multi-dimensional data by selecting the first time step
    if 'time' in swh_da.dims:
        try:
            swh_slice = swh_da.isel(time=0)
        except IndexError:
             print("! swh variable has time dimension but no time steps.")
             swh_slice = swh_da # Use the whole array if no time dimension or index 0 fails
        except Exception as e:
            print(f"! Error slicing swh data by time: {e}")
            swh_slice = swh_da
    else:
        swh_slice = swh_da

    # Ensure swh_slice has lat and lon dimensions
    if lat_coord not in swh_slice.dims or lon_coord not in swh_slice.dims:
         print(f"! swh variable does not have expected dimensions ({lat_coord}, {lon_coord}).")
         return None

    # Extract values, handling potential scalar data
    try:
        swh_values = np.squeeze(swh_slice.values)
    except Exception:
        swh_values = swh_slice.values


    # Check for fill/sentinel values
    fill = swh_da.attrs.get('_FillValue', None) or swh_da.encoding.get('_FillValue', None) or swh_da.attrs.get('missing_value', None) or swh_da.encoding.get('missing_value', None)

    min_dist = np.inf
    best_i, best_j = None, None

    # Search in expanding squares around the initial point
    for r in range(max_search + 1):
        for di in range(-r, r + 1):
            # Terdapat typo di skrip asli Anda (r+4), seharusnya (r+1)
            for dj in range(-r, r + 1): # <-- Koreksi dari r+4 menjadi r+1
                i = lat_idx_init + di
                j = lon_idx_init + dj

                # Check bounds
                if 0 <= i < len(lats) and 0 <= j < len(lons):
                    # Check if the grid cell has valid swh data
                    try:
                        # Access the value, handling potential multi-dimensional array if squeeze failed
                        if swh_values.ndim == 2:
                             val = swh_values[i, j]
                        elif swh_values.ndim == 1 and swh_values.shape[0] == len(lats) * len(lons):
                             # Attempt to flatten and then index if it matches grid size
                             val = swh_values[i * len(lons) + j]
                        else:
                             # If shape is unexpected, try accessing with isel
                            try:
                                # Ambil nilai pertama jika ada dimensi waktu
                                val_data = swh_da.isel({lat_coord: i, lon_coord: j})
                                if 'time' in val_data.dims:
                                    val = float(val_data.values[0])
                                else:
                                    val = float(val_data.values)
                            except Exception:
                                 continue


                        is_valid = np.isfinite(val)
                        if fill is not None:
                            is_valid = is_valid and (np.abs(val - fill) > 0)
                        if np.abs(val) >= 1e30: # Catch large sentinel values not explicitly marked
                             is_valid = False


                        if is_valid:
                            current_lat = lats[i]
                            current_lon = lons[j]
                            dist = sqrt((current_lat - target_lat)**2 + (current_lon - target_lon)**2)
                            if dist < min_dist:
                                min_dist = dist
                                best_i, best_j = i, j
                                # If we found a valid cell at the initial location (r=0), we can stop
                                if r == 0:
                                    return best_i, best_j, min_dist

                    except IndexError:
                        # Should not happen with bounds check, but just in case
                        pass
                    except Exception as e:
                         # print(f"! Error accessing/checking swh value at ({i},{j}): {e}")
                         continue

        # If a valid cell was found in this ring, return the best one found so far
        if best_i is not None:
             return best_i, best_j, min_dist


    # If no valid cell found after searching
    return None

print("Sel 1: Kelas ERA5WindRose dan semua fungsi berhasil dimuat.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.9 MB/s eta 0:00:00
Sel 1: Kelas ERA5WindRose dan semua fungsi berhasil dimuat.


In [2]:
# --- Membuat Widget GUI ---

# 1. Tombol Radio untuk Sektor Arah
radio_sektor = widgets.RadioButtons(
    options=['Omnidirectional', 'Sektoral'],
    value='Omnidirectional', # Nilai default
    description='Sektor Arah:',
    disabled=False
)

# 2. Input Teks untuk Sudut (dinonaktifkan secara default)
input_sudut_awal = widgets.FloatText(
    value=None,
    placeholder='Contoh: 350.0',
    description='Sudut Awal (°):',
    disabled=True # Nonaktif karena default-nya Omnidirectional
)

input_sudut_akhir = widgets.FloatText(
    value=None,
    placeholder='Contoh: 20.0',
    description='Sudut Akhir (°):',
    disabled=True # Nonaktif karena default-nya Omnidirectional
)

# 3. Tombol untuk Menjalankan Analisis
run_button = widgets.Button(
    description='Jalankan Analisis',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Klik untuk memulai pemrosesan data',
    icon='cogs' # (Gears)
)

# 4. Area Output untuk Menampilkan Log/Pesan
output_area = widgets.Output()

# 5. Mengatur Tata Letak GUI
gui_layout = widgets.VBox([
    radio_sektor,
    input_sudut_awal,
    input_sudut_akhir,
    widgets.HTML("<hr>"), # Garis pemisah
    run_button,
    output_area
])

print("Sel 2: Widget GUI berhasil dibuat (belum ditampilkan).")

Sel 2: Widget GUI berhasil dibuat (belum ditampilkan).


In [3]:
# --- Mendefinisikan Logika untuk GUI ---

def handle_radio_change(change):
    """Fungsi ini dipanggil saat nilai radio_sektor berubah."""
    if change['new'] == 'Sektoral':
        # Aktifkan input sudut jika Sektoral dipilih
        input_sudut_awal.disabled = False
        input_sudut_akhir.disabled = False
    else:
        # Nonaktifkan input sudut jika Omnidirectional dipilih
        input_sudut_awal.disabled = True
        input_sudut_akhir.disabled = True
        input_sudut_awal.value = None # Hapus nilai
        input_sudut_akhir.value = None # Hapus nilai

def on_run_button_clicked(b):
    """Fungsi ini dipanggil saat tombol 'Jalankan Analisis' diklik."""

    # 1. Bersihkan output sebelumnya dan tampilkan log di area output
    with output_area:
        output_area.clear_output()
        print("Memulai Analisis...")

        # 2. Ambil parameter dari skrip asli Anda
        # (Anda bisa mengubah ini jika diperlukan)
        lat, lon = -6.938011464344723, 110.40905826829982
        start_date = "2023-01-01"
        end_date = "2023-02-28"
        convert_to_cms = False
        content_dir = "/content/drive/MyDrive/DATA ERA 5/"
        out_dir = "output"
        os.makedirs(out_dir, exist_ok=True)

        # 3. Logika filter file (sama seperti skrip Anda)
        start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d")

        try:
            all_nc_files_in_dir = [f for f in os.listdir(content_dir) if f.lower().endswith('.nc')]
        except FileNotFoundError:
            print(f"X Error: Directory tidak ditemukan: {content_dir}")
            print("Pastikan Google Drive Anda ter-mount dan path-nya benar.")
            return

        all_nc_files_in_dir.sort()

        filtered_nc_paths = []
        for fname in all_nc_files_in_dir:
            match = re.search(r'(\d{6})\.nc$', fname)
            if match:
                file_yyyymm = match.group(1)
                file_year = int(file_yyyymm[:4])
                file_month = int(file_yyyymm[4:])
                file_date_start_of_month = datetime.datetime(file_year, file_month, 1)
                file_date_end_of_month = (file_date_start_of_month.replace(day=28) + datetime.timedelta(days=4)).replace(day=1) - datetime.timedelta(days=1)
                if file_date_start_of_month <= end_dt and file_date_end_of_month >= start_dt:
                    filtered_nc_paths.append(os.path.join(content_dir, fname))

        nc_files = filtered_nc_paths

        if not nc_files:
            print("X Error: Tidak ada file .nc yang relevan ditemukan untuk rentang tanggal yang ditentukan.")
            print(f" - Path: {content_dir}")
            print(f" - Rentang: {start_date} s/d {end_date}")
            return

        print(f"File(s) yang akan digunakan: {nc_files}")

        # 4. Jalankan pipeline analisis (LOAD, COMPUTE)
        try:
            wr = ERA5WindRose(nc_files, lat, lon, start_date, end_date, convert_to_cms=convert_to_cms)
            wr.load_dataset()
            wr.compute_wind()
            wr.compute_waveheight()
        except Exception as e:
            print(f"\n--- X GAGAL saat Load/Compute ---")
            print(f"Error: {e}")
            return

        # 5. --- LOGIKA BARU: FILTER SEKTORAL ---
        print("\n--- Menerapkan Filter Arah ---")
        mode = radio_sektor.value

        if mode == 'Sektoral':
            awal = input_sudut_awal.value
            akhir = input_sudut_akhir.value

            # Validasi input
            if awal is None or akhir is None or not np.isfinite(awal) or not np.isfinite(akhir):
                print("X Error: Untuk mode Sektoral, 'Sudut Awal' dan 'Sudut Akhir' harus diisi dengan angka yang valid.")
                return

            print(f"Mode Sektoral dipilih. Memfilter data antara {awal}° dan {akhir}°.")

            original_count = len(wr.df)

            # Cek jika sudut wrap-around (misal: 350 ke 20)
            if awal <= akhir:
                # Kasus normal (misal: 90 ke 180)
                mask = (wr.df['direction'] >= awal) & (wr.df['direction'] <= akhir)
            else:
                # Kasus wrap-around (misal: 350 ke 20)
                mask = (wr.df['direction'] >= awal) | (wr.df['direction'] <= akhir)

            wr.df = wr.df[mask].copy() # Filter DataFrame!

            if wr.df.empty:
                print(f"X Peringatan: Tidak ada data ditemukan dalam rentang sudut {awal}° - {akhir}°. Analisis dihentikan.")
                return
            else:
                print(f"Data berhasil difilter. Sisa data: {len(wr.df)} dari {original_count} baris.")

        else:
            print("Mode Omnidirectional dipilih. Menggunakan semua data (0-360°).")

        # 6. Lanjutkan pipeline (STATISTIK, PLOT, SAVE)
        print("\n--- Memulai Statistik dan Output ---")
        try:
            wr.generate_statistics()
            wr.generate_wave_statistics()

            # Tentukan nama file output unik berdasarkan mode
            suffix = f"_sektoral_{awal}_{akhir}" if mode == 'Sektoral' else "_omnidirectional"

            wr.plot_wind_rose(os.path.join(out_dir, f"wind_rose{suffix}.png"))
            wr.plot_wave_rose(os.path.join(out_dir, f"wave_rose{suffix}.png"))
            wr.plot_gumbel_return_values(os.path.join(out_dir, f"gumbel_return{suffix}"))
            wr.save_to_excel(os.path.join(out_dir, f"analysis_stats{suffix}.xlsx"))
            wr.save_extreme_values_to_new_excel(os.path.join(out_dir, f"extreme_values{suffix}.xlsx"))

            print("\n--- ✔️ Analisis Selesai Sukses ---")
            print(f"Semua file output untuk mode '{mode}' telah disimpan di folder '{out_dir}'.")

        except Exception as e:
            print(f"\n--- X GAGAL saat Statistik/Plot/Save ---")
            print(f"Error: {e}")
            print("Pastikan ada data yang tersisa setelah filtering.")


# --- Menghubungkan fungsi ke widget ---
radio_sektor.observe(handle_radio_change, names='value')
run_button.on_click(on_run_button_clicked)

print("Sel 3: Logika GUI berhasil didefinisikan dan dihubungkan.")

Sel 3: Logika GUI berhasil didefinisikan dan dihubungkan.


In [7]:
# Tampilkan GUI
display(gui_layout)